# Rozpoznawanie ręcznie pisanych cyfr za pomocą TensorFlow

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Pracę rozpoczniemy od pobrania zbioru MNIST, zawierającego 70 tyś. przykładów ręcznie pisanych cyfr 0-9 w formie obrazków 28x28 pikseli. Wykorzystamy w tym celu funkcję `fetch_mldata`, która pobera dane z serwisu [http://mldata.org/](mldata.org).

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

Poniższy kod dzieli losow dane na zbiory: uczący, walidujący i testowy w proporcji 70%/10%/20%. Wykorzystaj tak podzielone dane w dalszych krokach.

In [ ]:
n, p = mnist.data.shape
k = 10 # liczba klas
n_train = int(.7*n)
n_validation = int(.1*n)
indices = np.random.permutation(n)
train_indices = indices[:n_train]
validation_indices = indices[n_train:n_train+n_validation]
test_indices = indices[n_train+n_validation:]
X_train, y_train = mnist.data[train_indices,:], mnist.target[train_indices]
X_validation, y_validation = mnist.data[validation_indices,:], mnist.target[validation_indices]
X_test, y_test = mnist.data[test_indices,:], mnist.target[test_indices]

## Regresja logitstyczna

Stworzymy prosty model regresji logistycznej, uczony na surowych pikselach obrazków.

In [ ]:
tf.reset_default_graph()

`W` to macierz $p\times k$ (tzn. $p$ wag dla każdej z $k$ klas) zawierająca wagi cech (tutaj: pikseli obrazu), `b` to wyrazy wolne, różne dla każdej z `k` klas.

In [ ]:
W = tf.get_variable(dtype=tf.float32, name="W", shape=(p,k))
b = tf.get_variable(dtype=tf.float32, name="b", shape=(k,))

Przygotwujemy placeholdery na cechy `X_pl` i poprawne etykiety `y_pl`

In [ ]:
X_pl = tf.placeholder(dtype=tf.float32, shape=(None, p))
y_pl = tf.placeholder(dtype=tf.int64, shape=(None,))

Obliczamy `logits`, tzn. wyjście regresji liniowej **przed** zastosowaniem funkcji softmax: każdy wiersz odpowiada jednemu przykładowi z `X_pl`, a każda kolumna jednej z klas. Gdyby każdy z wierszy unormować funkcją softmax, to w kolejnych wierszach byłyby prawdopodobieństwa, że obiekt należy do danej klasy.

In [ ]:
logits = X_pl@W+b

Obliczamy *cross entropy* dla każdego z przykładów (tzn. `ent` jest wektorem). Korzystamy z funkcji `sparse_softmax_cross_entropy_with_logits`, która jako argument `labels` przyjmuje "normalne" etykiety klas, a jako `logits` nieunormowane prawdopodobieństwa (w postaci jak opisana wyżej). Na bazie `ent` obliczamy średnią i otrzymujemy stratę `loss`, którą optymailzujemy za pomocą `AdamOptimizer`.

In [ ]:
ent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_pl, logits=logits)
loss = tf.reduce_mean(ent)
minimizer = tf.train.AdamOptimizer().minimize(loss)

Obliczamy wektor `y_pred` przez wybranie kolumny zawierającej największą wartość w każdym wierszu (*softmax* jest funkcją monotoniczną, więc do zrobienia predykcji nie potrzebujemy prawdopodobieństw - wystarczy wybrać największą wartość). Następnie obliczamy *accuracy*: 
1. tworzymy wektor `correct`, który zawiera `True` w tych komórkach, gdzie odpowiadające komórki `y_pred` i `y_pl` są równe oraz `False` w przeciwnym razie;
2. rzutujemy wektor `correct` na liczby zmiennoprzecinkowe (`True` zamienia się w `1.0`, a `False`, w `0.0`);
3. uśredniamy, żeby obliczyć *accuracy*.

In [ ]:
y_pred = tf.argmax(logits, axis=1)
correct = tf.equal(y_pred, y_pl)
correct = tf.cast(correct, dtype=tf.float32)
acc = tf.reduce_mean(correct)

Dalej jest już tak samo jak przy regresji liniowej: tworzymy operator do inicjalizacji zmiennych i obiekt sesji, inicjalizujemy zmienne i przez `n_epoch` trenujemy na batchach rozmiaru `batch_size`, zbierając wartości *loss* i *accuracy* na zbiorze uczącym.

In [ ]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

In [ ]:
sess.run(init)
loss_values = []
acc_values = []
batch_size = 100
n_epoch = 2000

In [ ]:
for epoch in range(n_epoch):
    indices = np.random.choice(n_train, size=batch_size) 
    feed_dict = {X_pl: X_train[indices,:], y_pl: y_train[indices]}
    _, loss_value = sess.run([minimizer, loss], feed_dict)
    loss_values.append(loss_value)
    if epoch % 100 == 0:
        acc_val = sess.run(acc, {X_pl: X_validation, y_pl: y_validation})    
        acc_values.append(acc_val)

In [ ]:
plt.title("Loss na zbiorze uczącym")
plt.plot(loss_values)
plt.show()
plt.title("Accuracy na zbiorze walidującym (co 100 epok)")
plt.plot(acc_values)
plt.show()
print(acc_values[-1])

## Early stopping

Zadanie do samodzielnego wykonania: wykorzystaj powyższy kod i zaimplementuj *early stopping* zamiast trenowania przez stałą liczbę epok.

In [ ]:
# Tu kod, może być w więcej niż jednej komórce

## Sieć neuronowa z jedną warstwą ukrytą

Regresja logistyczna to wariant sieci neuronowej bez warstw ukrytych. Wykorzystaj powyższy kod i rozszerz go o warstwę ukrytą o h=500 neuronach.

Potrzebujesz następujących zmiennych (wszystkie w typie zmiennoprzecinkowym):
* `W1` macierz rozmiaru $p\times h$
* `b1` wektor rozmiaru $h$
* `W2` macierz rozmiaru $h\times k$
* `b2` wektor rozmiaru $k$

Logits mają zostać obliczone zgodnie z poniższym wzorem:
$$ logits = elu(XW_1+b_1)W_2+b_2 $$
gdzie funkcja $elu$ jest funkcją wprowadzającą nieliniowość zaimplementowaną w TensorFlow jako `tf.nn.elu`:
$$ elu(x) = \begin{cases} e^x - 1 & x<0 \\ x & x\geq 0 \end{cases} $$

Zaimplementuj early stopping. W trakcie uczenia zbieraj wartości straty i accuracy na zbiorze uczącym i na zbiorze walidującym, a następnie narysuj je na wykresach. Jeżeli uczenie trwa zbyt wolno jak na Twój gust, zmniejsz `batch_size` i/lub liczbę neuronów w warstwie ukrytej.

In [ ]:
h = 500

tf.reset_default_graph()

In [ ]:
# Tu kod